In [1]:
using CSV, DataFrames, Statistics, Random

In [2]:
df = CSV.read("Dataset_With_Payoff.csv", DataFrame)

Row,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Required_Bandwidth_Mbps,Allocated_Bandwidth_Mbps,Latency_ms,Signal_Strength_dBm,alpha_j,Payoff
,String15,String15,String31,String15,String7,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,9/3/2023 10:00,User_1,Video_Call,-75 dBm,30 ms,10 Mbps,15 Mbps,0.7,10.0,15.0,30.0,-75.0,0.41204,9.04615
2,9/3/2023 10:00,User_2,Voice_Call,-80 dBm,20 ms,100 Kbps,120 Kbps,0.8,0.1,0.12,20.0,-80.0,0.00573338,0.00135099
3,9/3/2023 10:00,User_3,Streaming,-85 dBm,40 ms,5 Mbps,6 Mbps,0.75,5.0,6.0,40.0,-85.0,0.091874,0.822168
4,9/3/2023 10:00,User_4,Emergency_Service,-70 dBm,10 ms,1 Mbps,1.5 Mbps,0.9,1.0,1.5,10.0,-70.0,0.139614,0.471199
5,9/3/2023 10:00,User_5,Online_Gaming,-78 dBm,25 ms,2 Mbps,3 Mbps,0.85,2.0,3.0,25.0,-78.0,0.163073,0.947918
6,9/3/2023 10:00,User_6,Background_Download,-90 dBm,50 ms,500 Kbps,550 Kbps,0.7,0.5,0.55,50.0,-90.0,0.0110257,0.00752506
7,9/3/2023 10:00,User_7,Web_Browsing,-88 dBm,30 ms,1 Mbps,1 Mbps,0.6,1.0,1.0,30.0,-88.0,0.03592,0.0501574
8,9/3/2023 10:00,User_8,IoT_Temperature,-95 dBm,100 ms,10 Kbps,15 Kbps,0.5,0.01,0.015,100.0,-95.0,4.31731e-5,9.93471e-8
9,9/3/2023 10:00,User_9,Video_Streaming,-82 dBm,35 ms,3 Mbps,3.5 Mbps,0.8,3.0,3.5,35.0,-82.0,0.0867229,0.504964


In [3]:
bw_low = 0.1             
lat_high = 100.0         
total_bandwidth = 1000.0 

1000.0

In [4]:
max_latency = maximum(df.Latency_ms)
max_signal = maximum(df.Signal_Strength_dBm)

-40.0

In [5]:
function compute_payoff(alpha, bandwidth, latency, signal, resource, max_latency, max_signal)
    return alpha * bandwidth * (1 - latency / max_latency) * (1 + signal / max_signal) * resource
end

compute_payoff (generic function with 1 method)

In [6]:
function get_allocated_bandwidth(requested_bandwidths)
    total_requested = sum(requested_bandwidths)
    return total_requested == 0 ? zeros(length(requested_bandwidths)) :
           (total_bandwidth / total_requested) * requested_bandwidths
end

get_allocated_bandwidth (generic function with 1 method)

In [7]:
function compute_expected_payoff(i, p, df, max_latency, max_signal, n_samples=500)
    n_players = nrow(df)
    payoff_sum = 0.0

    for _ in 1:n_samples
        requested = zeros(n_players)
        latencies = zeros(n_players)

        for j in 1:n_players
            bw_i = df.Required_Bandwidth_Mbps[j]
            lat_i = df.Latency_ms[j]
            if rand() < p[j]
                requested[j] = bw_i
                latencies[j] = lat_i
            else
                requested[j] = bw_low
                latencies[j] = lat_high
            end
        end

        allocated = get_allocated_bandwidth(requested)
        payoff = compute_payoff(
            df.alpha_j[i],
            allocated[i],
            latencies[i],
            df.Signal_Strength_dBm[i],
            df.Resource_Allocation[i],
            max_latency,
            max_signal
        )
        payoff_sum += payoff
    end

    return payoff_sum / n_samples
end

compute_expected_payoff (generic function with 2 methods)

In [8]:
function best_response_dynamics(df; max_iter=1000, eta=0.5, ε=1e-5)
    n_players = nrow(df)
    p = rand(n_players)
    max_latency = maximum(df.Latency_ms)
    max_signal = maximum(df.Signal_Strength_dBm)

    for iter in 1:max_iter
        p_old = copy(p)
        max_change = 0.0

        for i in 1:n_players
            payoff_hi = compute_expected_payoff(i, p, df, max_latency, max_signal)
            p_test = copy(p)
            p_test[i] = 0.0
            payoff_lo = compute_expected_payoff(i, p_test, df, max_latency, max_signal)

            best_response = payoff_hi > payoff_lo ? 1.0 : 0.0
            p[i] = (1 - eta) * p[i] + eta * best_response

            max_change = max(max_change, abs(p[i] - p_old[i]))
        end

        if max_change < ε
            println("Converged after $iter iterations.")
            return p
        end
    end

    println("Max iterations reached.")
    return p
end


best_response_dynamics (generic function with 1 method)

In [ ]:
Random.seed!(123)
mixed_strategies = best_response_dynamics(df)